In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
#plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 결측치 시각화를 위한 라이브러리
import missingno

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

pd.set_option('display.max_rows', 1000)

In [3]:
member_d = pd.read_csv('/Users/kimgayeong/Desktop/파이널/작업/train_회원정보.csv')
member_d.head()
member_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 71 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   기준년월                int64 
 1   ID                  object
 2   남녀구분코드              int64 
 3   연령                  object
 4   Segment             object
 5   회원여부_이용가능           int64 
 6   회원여부_이용가능_CA        int64 
 7   회원여부_이용가능_카드론       int64 
 8   소지여부_신용             int64 
 9   소지카드수_유효_신용         int64 
 10  소지카드수_이용가능_신용       int64 
 11  입회일자_신용             int64 
 12  입회경과개월수_신용          int64 
 13  회원여부_연체             int64 
 14  이용거절여부_카드론          int64 
 15  동의여부_한도증액안내         int64 
 16  수신거부여부_TM           int64 
 17  수신거부여부_DM           int64 
 18  수신거부여부_메일           int64 
 19  수신거부여부_SMS          int64 
 20  탈회횟수_누적             int64 
 21  최종탈회후경과월            int64 
 22  탈회횟수_발급6개월이내        int64 
 23  탈회횟수_발급1년이내         int64 
 24  거주시도명               object
 25  마케팅동의여부           

In [5]:
credit_d = pd.read_csv('/Users/kimgayeong/Desktop/파이널/작업/train_신용정보.csv')
credit_d.head()
credit_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 40 columns):
 #   Column          Dtype  
---  ------          -----  
 0   기준년월            int64  
 1   ID              object 
 2   최초한도금액          int64  
 3   카드이용한도금액        int64  
 4   CA한도금액          int64  
 5   일시상환론한도금액       int64  
 6   월상환론한도금액        int64  
 7   CA이자율_할인전       float64
 8   CL이자율_할인전       float64
 9   RV일시불이자율_할인전    float64
 10  RV현금서비스이자율_할인전  float64
 11  RV약정청구율         float64
 12  RV최소결제비율        float64
 13  자발한도감액횟수_R12M   object 
 14  자발한도감액금액_R12M   int64  
 15  자발한도감액후경과월      int64  
 16  강제한도감액횟수_R12M   int64  
 17  강제한도감액금액_R12M   int64  
 18  강제한도감액후경과월      int64  
 19  한도증액횟수_R12M     object 
 20  한도증액금액_R12M     int64  
 21  한도증액후경과월        int64  
 22  상향가능한도금액        int64  
 23  상향가능CA한도금액      int64  
 24  카드론동의여부         object 
 25  월상환론상향가능한도금액    int64  
 26  일시불ONLY전환가능여부   int64  
 27  카드이용한도금액_B1M    int64  
 28  카드이용한도금액_B2M

# 회원정보

In [16]:
# 1. Segment를 숫자로 변환 (예: A=0, B=1, ..., E=4)
member_d['Segment_num'] = member_d['Segment'].astype('category').cat.codes

# 2. Segment_num을 제외한 수치형 컬럼 추출
numeric_cols = member_d.select_dtypes(include=['number']).columns.drop('Segment_num')

# 3. 각 수치형 변수와 Segment_num 간 상관계수 계산
correlations = member_d[numeric_cols].corrwith(member_d['Segment_num'])

# 4. 정렬 및 데이터프레임 변환
cor_df = correlations.to_frame(name='상관계수').sort_values(by='상관계수', ascending=False)

# 5. 지수 표기 없이 소수점 7자리로 출력
with pd.option_context('display.float_format', '{:.7f}'.format):
    display(cor_df)

,상관계수
입회일자_신용,0.2522895
탈회횟수_누적,0.0687440
남녀구분코드,0.0603231
마케팅동의여부,0.0477100
카드신청건수,0.0389222
이용거절여부_카드론,0.0364000
탈회횟수_발급1년이내,0.0323531
탈회횟수_발급6개월이내,0.0155203
회원여부_이용가능_카드론,0.0136477
청구금액_기본연회비_B0M,0.0101168


In [20]:
for col in ['이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M',
            '할인금액_제휴연회비_B0M']:
    nunique = member_d[col].nunique(dropna=True)
    missing = member_d[col].isnull().sum()
    print(f"{col} → 고유값 개수: {nunique}, 결측치 수: {missing}")

이용카드수_체크_가족 → 고유값 개수: 1, 결측치 수: 0
이용금액_R3M_체크_가족 → 고유값 개수: 1, 결측치 수: 0
연회비할인카드수_B0M → 고유값 개수: 1, 결측치 수: 0
할인금액_기본연회비_B0M → 고유값 개수: 1, 결측치 수: 0
할인금액_제휴연회비_B0M → 고유값 개수: 1, 결측치 수: 0


## 제거할지?

# 신용정보

In [34]:
# 필요한 컬럼만 추출해서 df2를 준비
member_df_reduced = member_d[['ID', '기준년월', 'Segment']]

# ID와 기준년월 기준으로 병합 (inner 또는 left join 가능)
balance_df_merged = pd.merge(credit_d, member_df_reduced, on=['ID', '기준년월'], how='left')

In [36]:
balance_df_merged

,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일,Segment
0,201810,TRAIN_000000,0,19724,5990,0,0,22.995321,18.481194,18.121342,...,0회,0,3,3,0,0,0,0,99999999,D
1,201810,TRAIN_000001,0,9997,4643,42214,92292,14.809244,15.522862,10.793073,...,0회,0,3,3,0,0,0,0,322,E
2,201810,TRAIN_000002,0,86125,32485,0,0,21.934388,18.367193,17.680993,...,0회,0,3,3,0,0,0,0,2374,C
3,201810,TRAIN_000003,0,19048,10065,0,0,22.999954,22.999954,19.210532,...,0회,0,3,3,0,0,0,0,99999999,D
4,201810,TRAIN_000004,0,176764,52253,47996,156564,14.700813,11.898927,10.267081,...,0회,0,3,3,0,0,0,0,99999999,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201811,TRAIN_399995,0,20677,7715,0,54676,15.204317,11.899371,10.402842,...,0회,0,3,3,0,0,0,0,99999999,E
2399996,201811,TRAIN_399996,0,85594,30880,0,156200,14.815354,15.628568,11.022244,...,0회,0,3,3,0,0,0,0,99999999,D
2399997,201811,TRAIN_399997,0,53574,18017,0,0,16.961238,16.292944,13.679558,...,0회,0,3,3,0,0,0,0,99999999,C
2399998,201811,TRAIN_399998,0,10003,2936,90000,180887,15.177413,11.900715,10.549119,...,0회,0,3,3,0,0,0,0,99999999,E
